In [5]:
import numpy as np
from gym_env.envs import ground_env as ge
from stable_baselines3.common.env_checker import check_env
from stable_baselines3.common.vec_env import VecFrameStack
from stable_baselines3 import DQN, A2C, PPO
import gym
import os
import glob
import random
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from collections import Counter

In [2]:
grid_shape = 100

rewards_dict = {'cosine_distance_scale': 100, 'penalty_for_one_step': 0.01,
 'penalty_for_one_step_when_ball_is_here': 0.1, 'reward_for_catch':100, 'time_between_catch_next_ball':100,
  'game_end_total':1000,'game_end_pickup_time':10000}
length_of_test = 10*1e3
  
reward_params = list(rewards_dict.values())
env = ge.playground_env(100,'center','uniform',reward_params =  reward_params)
model = DQN("MlpPolicy", env, verbose=0, tensorboard_log="sac",learning_rate=0.001)
model.learn(total_timesteps=1*1e5, log_interval=4,)
model.save("models/station_policy")



In [7]:

model =  DQN.load('models/station_policy')

reward_params_test = reward_params
reward_params_test[-1] = length_of_test
n = 5
env = {}
obs = {}
done = {}
for i in range(n):
    env[i] = ge.playground_env(grid_shape,'center','uniform',reward_params = reward_params)
    obs[i] = env[i].reset()
    done[i] = False
done_dict = Counter(done.values())
action_profile = []
reward_profile = []
frames_ = []
X_station = []
Y_station = []
c = 0
from collections import Counter


while not (done_dict[True] == 5):
    c = c +1 
    x = []
    y =[]
    text_size = []
    text_symbol = []
    for i in range(n):
        if done[i] != True:
            act = model.predict(obs[i])
            obs_, reward, done[i], info = env[i].step(act[0])
            obs[i] = obs_
            
            if env[i].PG.generated:
                x.append(env[i].PG.pos[0])
                x.append(env[i].PG.ran_pos[0])
                y.append(env[i].PG.pos[1])
                y.append(env[i].PG.ran_pos[1])
                text_size = text_size +[20,20]
                text_symbol = text_symbol +['🚚','📦']
            else:
                x.append(env[i].PG.pos[0])
                y.append(env[i].PG.pos[1])
                text_size = text_size +[20]
                text_symbol = text_symbol +['🚛']
    frames_.append(go.Frame(data=[go.Scatter(x=x, y=y,  mode='text',textfont=dict(size=text_size), text=text_symbol )]))
    done_dict = Counter(done.values())


In [8]:
from turtle import color
import plotly.graph_objects as go
fig = go.Figure(
    data=[go.Scatter(mode='text',textfont=dict(size=[20, 20]), text=['🚶','⚽️'])],
    layout=go.Layout(
        xaxis=dict(range=[0, grid_shape], autorange=False),
        yaxis=dict(range=[0, grid_shape], autorange=False),
        title="Truck Pickup Optimization MultiRL Agent",
        updatemenus=[dict(buttons = [dict(
                                        args = [None, {"frame": {"duration": 1, 
                                                                "redraw": False},
                                                        "fromcurrent": True, 
                                                        "transition": {"duration": 0}}],
                                        label = "Play",
                                        method = "animate")],
                                type='buttons',
                                showactive=False)]
    ),
    frames=frames_
)
fig.update_layout(width= 600, height= 600)
fig.update_layout(showlegend=False)
fig.update_xaxes(visible=False)
fig.update_yaxes(visible=False)
fig.show()

In [65]:
not (done_dict[True] == 5)

True

In [62]:
done_dict[True]

0